In [ ]:
import sagemaker
import boto3
from sagemaker.session import Session
from sagemaker import get_exeution_role


## Cargar y Preprocesar los Datos

In [30]:
import pandas as pd
import sagemaker
import s3fs
import numpy as np
import boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [31]:
from sagemaker.session import Session
from sagemaker import get_exeution_role

ImportError: cannot import name 'get_exeution_role' from 'sagemaker' (/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/__init__.py)

In [32]:
sagemaker_session = Session()
boto_sess = boto3.Session()

role = get_execution_role()
default_bucket = sagemaker_session.default_bucket()

sm = boto_sess.client("sagemaker")
region = boto_sess.region_name

In [36]:
#covid_data = pq.ParquetDataset('s3://aws-bucket-lab1/covid19.parquet', filesystem=s3).read_pandas().to_pandas()
covid_data=pd.read_parquet('s3://aws-bucket-lab1/covid19.parquet',engine="pyarrow")
covid_data.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [37]:
# Preprocesar los datos
# Convertir 'DATE_DIED' a valores binarios
covid_data['DATE_DIED'] = covid_data['DATE_DIED'].apply(lambda x: 0 if x == '9999-99-99' else 1)

# Separar características y etiqueta
X = covid_data.drop('DATE_DIED', axis=1)
y = covid_data['DATE_DIED']

In [38]:
# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalización de los datos (si es necesario)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Crear el Modelo de Clasificación con Keras

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [40]:
# Definir el modelo
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

In [41]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## Entrenar y Evaluar el Modelo

In [42]:
# Entrenar el modelo
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32)

Epoch 1/10
26215/26215 [==============================] - 52s 2ms/step - loss: 0.1344 - accuracy: 0.9368
Epoch 2/10
26215/26215 [==============================] - 51s 2ms/step - loss: 0.1273 - accuracy: 0.9392
Epoch 3/10
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1259 - accuracy: 0.9402
Epoch 4/10
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1252 - accuracy: 0.9404
Epoch 5/10
26215/26215 [==============================] - 51s 2ms/step - loss: 0.1248 - accuracy: 0.9406
Epoch 6/10
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1244 - accuracy: 0.9408
Epoch 7/10
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1243 - accuracy: 0.9409
Epoch 8/10
26215/26215 [==============================] - 51s 2ms/step - loss: 0.1240 - accuracy: 0.9410
Epoch 9/10
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1238 - accuracy: 0.9413
Epoch 10/10
26215/26215 [==============================

In [43]:
# Evaluar el modelo
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Precisión del modelo: {accuracy}')

6554/6554 [==============================] - 8s 1ms/step - loss: 0.1228 - accuracy: 0.9414
Precisión del modelo: 0.941372811794281
